# Chapter 5 Python, Django, Pandasを用いた気象データ分析結果のWeb公開

## 1. Django Web アプリケーションの作成
まずはDjangoの環境を設定します。
前 Chapter にならって Django のインストールからアプリケーションの作成まで実施します。

In [ ]:
# Django のインストール
!pip install django

# Chapter 3 の内容のデータを取得する。
!pip install pandas
!pip install openmeteo-requests
!pip install requests-cache retry-requests

# 描画のために django-pandas と plotly をインストール
!pip install django-pandas plotly

# Django のプロジェクト作成
!django-admin startproject mysite

# myapp アプリケーションの作成
!cd mysite && python manage.py startapp myapp

# Django を GoogleColab 上で動作させるための準備
# Django のネットワークを設定する -> colab.research.google.com に設定する
# ALLOWED_HOSTS = ['colab.research.google.com']
!cd mysite/mysite && sed -i.bak -e 's/\(ALLOWED_HOSTS\).*/\1 = ["colab\.research\.google\.com"]/g' settings.py
!cd mysite/mysite && grep ALLOWED_HOSTS settings.py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 6.3 MB/s eta 0:00:00
ALLOWED_HOSTS = ["colab.research.google.com"]


In [ ]:
# プロキシの設定
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(8000)"))

# 環境設定
import os
from pathlib import Path

basedir = Path(os.getcwd())
appdir = basedir / "mysite" / "myapp"
configdir = basedir / "mysite" / "mysite"

https://g9nx0k02jci-496ff2e9c6d22116-8000-colab.googleusercontent.com/


## 2. 描画に必要なモデルを追加する。

*   気象データを投入する WeatherData モデルを作成する





In [ ]:
# models.py
code = """
from django.db import models

class WeatherData(models.Model):
    date = models.DateField()
    temperature = models.FloatField()
    sunshine_duration = models.FloatField()
"""
with open(appdir / "models.py", "w") as f:
  f.write(code)

## 3. WeatherData モデルをデータベースに反映させる

In [ ]:
# settings.py に myapp を追加する
!cd mysite/mysite && sed -i.bak "/INSTALLED_APPS = \[/a \    'myapp\.apps\.MyappConfig'," settings.py
!cd mysite && python manage.py makemigrations myapp
!cd mysite && python manage.py migrate

Migrations for 'myapp':
  myapp/migrations/0001_initial.py
    - Create model WeatherData
Operations to perform:
  Apply all migrations: admin, auth, contenttypes, myapp, sessions
Running migrations:
  Applying contenttypes.0001_initial... OK
  Applying auth.0001_initial... OK
  Applying admin.0001_initial... OK
  Applying admin.0002_logentry_remove_auto_add... OK
  Applying admin.0003_logentry_add_action_flag_choices... OK
  Applying contenttypes.0002_remove_content_type_name... OK
  Applying auth.0002_alter_permission_name_max_length... OK
  Applying auth.0003_alter_user_email_max_length... OK
  Applying auth.0004_alter_user_username_opts... OK
  Applying auth.0005_alter_user_last_login_null... OK
  Applying auth.0006_require_contenttypes_0002... OK
  Applying auth.0007_alter_validators_add_error_messages... OK
  Applying auth.0008_alter_user_username_max_length... OK
  Applying auth.0009_alter_user_last_name_max_length... OK
  Applying auth.0010_alter_group_name_max_length... OK
  A

## 4. Chapter 3 の内容にならいお天気のデータを取得し csv へ変換する。

In [ ]:
# Setup the Open-Meteo API client with cache and retry on error
import openmeteo_requests
import requests_cache
from retry_requests import retry
import pandas as pd

cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 35.66712,
	"longitude": 138.57361870057653,
	"start_date": "1973-01-02",
	"end_date": "2023-12-31",
	"daily": ["weather_code", "temperature_2m_max", "temperature_2m_min", "temperature_2m_mean", "apparent_temperature_max", "apparent_temperature_min", "apparent_temperature_mean", "sunrise", "sunset", "daylight_duration", "sunshine_duration", "wind_speed_10m_max", "wind_gusts_10m_max", "wind_direction_10m_dominant", "shortwave_radiation_sum"],
	"timezone": "Asia/Tokyo"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°E {response.Longitude()}°N")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process daily data. The order of variables needs to be the same as requested.
# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_weather_code = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_max = daily.Variables(1).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(2).ValuesAsNumpy()
daily_temperature_2m_mean = daily.Variables(3).ValuesAsNumpy()
daily_apparent_temperature_max = daily.Variables(4).ValuesAsNumpy()
daily_apparent_temperature_min = daily.Variables(5).ValuesAsNumpy()
daily_apparent_temperature_mean = daily.Variables(6).ValuesAsNumpy()
daily_sunrise = daily.Variables(7).ValuesAsNumpy()
daily_sunset = daily.Variables(8).ValuesAsNumpy()
daily_daylight_duration = daily.Variables(9).ValuesAsNumpy()
daily_sunshine_duration = daily.Variables(10).ValuesAsNumpy()
daily_wind_speed_10m_max = daily.Variables(11).ValuesAsNumpy()
daily_wind_gusts_10m_max = daily.Variables(12).ValuesAsNumpy()
daily_wind_direction_10m_dominant = daily.Variables(13).ValuesAsNumpy()
daily_shortwave_radiation_sum = daily.Variables(14).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s"),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s"),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}
daily_data["weather_code"] = daily_weather_code
daily_data["temperature_2m_max"] = daily_temperature_2m_max
daily_data["temperature_2m_min"] = daily_temperature_2m_min
daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
daily_data["apparent_temperature_max"] = daily_apparent_temperature_max
daily_data["apparent_temperature_min"] = daily_apparent_temperature_min
daily_data["apparent_temperature_mean"] = daily_apparent_temperature_mean
daily_data["sunrise"] = daily_sunrise
daily_data["sunset"] = daily_sunset
daily_data["daylight_duration"] = daily_daylight_duration
daily_data["sunshine_duration"] = daily_sunshine_duration
daily_data["wind_speed_10m_max"] = daily_wind_speed_10m_max
daily_data["wind_gusts_10m_max"] = daily_wind_gusts_10m_max
daily_data["wind_direction_10m_dominant"] = daily_wind_direction_10m_dominant
daily_data["shortwave_radiation_sum"] = daily_shortwave_radiation_sum

daily_dataframe = pd.DataFrame(data = daily_data)

# date列をDateTime型に変換
daily_dataframe['date'] = pd.to_datetime(daily_dataframe['date'])

# 年ごとにグループ化して平均を計算
yearly_averages = daily_dataframe.groupby(daily_dataframe['date'].dt.year).mean()

yearly_averages.to_csv("data.csv")

Coordinates 35.6766242980957°E 138.53282165527344°N
Elevation 280.0 m asl
Timezone b'Asia/Tokyo' b'JST'
Timezone difference to GMT+0 32400 s


<ipython-input-6-5104afbb297b>:78: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  yearly_averages = daily_dataframe.groupby(daily_dataframe['date'].dt.year).mean()


## 5. データを Django にインポートする

In [ ]:
# data.csv から必要なデータのみ取得する
df = pd.read_csv("data.csv")
df = df[["date", "temperature_2m_max", "sunshine_duration"]]
df = df.rename(columns={"temperature_2m_max": "temperature"})

In [ ]:
# django アプリケーションに必要なデータを投入する
import sys
import os
import django
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"


sys.path.append(str(basedir / "mysite"))
os.environ.setdefault("DJANGO_SETTINGS_MODULE","mysite.settings")

django.setup()

from myapp.models import WeatherData
from datetime import datetime

for i in df.itertuples():
  ps = WeatherData(date=datetime(year=i.date, month=1, day=1), temperature=i.temperature, sunshine_duration=i.sunshine_duration)
  ps.save()

## 6. ビューの作成
DjangoでWebページを表示するためのビューを作成します。views.pyを編集します。

ここで Django Pandas を使い、Django のデータから Pandas のデータフレームへ変更している。

**これも Python の強みの一つで必要そうな機能は誰かしらが実装している可能性があります。**

※ 自分で何でも作らずに Python のパッケージを探してみるのもいいことです。

In [ ]:


# views.py
code = """
from django.shortcuts import render
import plotly.graph_objects as go
import pandas as pd
import plotly.express as px
from django_pandas.io import read_frame
from .models import WeatherData

def index(request):
  datas = WeatherData.objects.all()
  df = read_frame(
    datas,
    fieldnames=[
      "date", "temperature", "sunshine_duration"
    ]
  )
  temp = px.bar(df, x="date", y=["temperature"], title="Temperature (C)", height=300)
  temp_g = weather.to_html(full_html=False, include_plotlyjs=False)

  return render(request, "index.html", {"weather": temp_g})
"""

with open(appdir / "views.py", "w") as f:
  f.write(code)

## 7. URLのルーティング
URLをビューに接続します。urls.pyを編集します。

In [ ]:
# urls.py
code = """
from django.urls import path
from myapp import views

urlpatterns = [
    path('', views.index, name='index'),
]
"""
with open(configdir / "urls.py", "w") as f:
  f.write(code)


## 7. テンプレートの作成
Webページの見た目を決めるテンプレートを作成します。templatesディレクトリにhtmlファイルを作成します。

In [ ]:
# template index.html
(appdir / "templates").mkdir(exist_ok=True)
(appdir / "templates" / "myapp").mkdir(exist_ok=True)

html = """
<html>
 <head>
   <title>Weather Graph</title>
   <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
   <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet" integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
 </head>
 <body>
 <div>
  <h1>Chapter 5. Python, Django, Pandasを用いた気象データ分析結果のWeb公開</h1>
 </div>
  <div>
    {% autoescape off %}

    {{ weather }}

    {% endautoescape %}
 </body>
</html>
"""

with open(appdir / "templates" / "index.html", "w") as f:
  f.write(html)

## 8. Djangoサーバーの起動
最後にDjangoのサーバーを起動して、Webサイトが正しく動作するか確認します。

In [ ]:
# Django サーバーの起動
!cd mysite && python manage.py runserver 8000

Watching for file changes with StatReloader
Performing system checks...

System check identified no issues (0 silenced).
January 24, 2024 - 10:27:53
Django version 5.0.1, using settings 'mysite.settings'
Starting development server at http://127.0.0.1:8000/
Quit the server with CONTROL-C.

[24/Jan/2024 10:27:55] "GET / HTTP/1.1" 200 9878
Not Found: /favicon.ico
[24/Jan/2024 10:27:55] "GET /favicon.ico HTTP/1.1" 404 2129
[24/Jan/2024 10:28:43] "GET / HTTP/1.1" 200 9878
Not Found: /favicon.ico
[24/Jan/2024 10:28:43] "GET /favicon.ico HTTP/1.1" 404 2129


# 演習
時間に余裕がある方は、以下の課題に取り組んでみましょう。

## 演習課題1: 日照時間 (sunshine_duration) のグラフを追加
* 目的: 日照時間のグラフを追加する
* 手順: views.py に sunshine_duration 用のグラフを追加する。
 * views.py に sunshine_duration 用のコードを追加する。
 * index.html にグラフを追加するコードを追加する

## 演習課題2: Chapter3 の収穫量のデータも投入する
* 目的: 収穫量と比較するため
* 手順:
 * models.py に YieldModel を追加する
 * YieldModel をデータベースに反映させる
 * データを投入する
 * views.py に収穫量用のコードを追加する
 * index.html にグラフを追加するコードを追加する

## 演習課題3: 自由なデータソースを使い自分好みのサイトを作ってみる
この後はみなさん好きな環境で実施しましょう。GoogleColab でも問題ないですし、VSCode で環境を作ってしまっても問題ないです。

